In [2]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, mannwhitneyu 
import seaborn as sns 
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
%matplotlib inline

/Users/anastasia/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df_original = pd.read_csv('Desktop/Project Austin/Austin_crime/Crime_Reports_aug_2020.csv')
df_original.head(3)

/Users/anastasia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,...,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location
0,20125003262,PROTECTIVE ORDER,3829,N,01/23/2012 05:17:00 AM,01/23/2012,517.0,01/23/2012 05:17:00 AM,01/23/2012,517.0,...,NaN,N,01/23/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20072832112,ASSAULT WITH INJURY,900,N,10/10/2007 09:00:00 PM,10/10/2007,2100.0,10/10/2007 10:00:00 PM,10/10/2007,2200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20081232700,FAMILY DISTURBANCE,3400,N,05/02/2008 11:11:00 PM,05/02/2008,2311.0,05/02/2008 11:11:00 PM,05/02/2008,2311.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## General Information

In [4]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2259648 entries, 0 to 2259647
Data columns (total 27 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Incident Number              int64  
 1   Highest Offense Description  object 
 2   Highest Offense Code         int64  
 3   Family Violence              object 
 4   Occurred Date Time           object 
 5   Occurred Date                object 
 6   Occurred Time                float64
 7   Report Date Time             object 
 8   Report Date                  object 
 9   Report Time                  float64
 10  Location Type                object 
 11  Address                      object 
 12  Zip Code                     float64
 13  Council District             float64
 14  APD Sector                   object 
 15  APD District                 object 
 16  PRA                          object 
 17  Census Tract                 float64
 18  Clearance Status             object 
 19  

In [5]:
nRow, nCol = df_original.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 2259648 rows and 27 columns


In [6]:
df = df_original.copy()
del df['Incident Number']
del df['Highest Offense Code']
del df['Clearance Status']
del df['Council District']
del df['APD Sector']
del df['Census Tract']
del df['UCR Category']
del df['APD District']
del df['PRA']
del df['Clearance Date']
del df['Category Description']

# Missing Values

In [7]:
df.isnull().sum()*100/df.isnull().count()

Highest Offense Description    0.000000
Family Violence                0.000000
Occurred Date Time             0.007302
Occurred Date                  0.000000
Occurred Time                  0.003717
Report Date Time               0.000044
Report Date                    0.000000
Report Time                    0.000044
Location Type                  0.923108
Address                        0.000531
Zip Code                       0.608590
X-coordinate                   0.435643
Y-coordinate                   0.435643
Latitude                       1.685926
Longitude                      1.685926
Location                       1.685926
dtype: float64

In [8]:
df[df['Zip Code'].isnull()]

,Highest Offense Description,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,Location Type,Address,Zip Code,X-coordinate,Y-coordinate,Latitude,Longitude,Location
0,PROTECTIVE ORDER,N,01/23/2012 05:17:00 AM,01/23/2012,517.0,01/23/2012 05:17:00 AM,01/23/2012,517.0,RESIDENCE / HOME,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASSAULT WITH INJURY,N,10/10/2007 09:00:00 PM,10/10/2007,2100.0,10/10/2007 10:00:00 PM,10/10/2007,2200.0,BAR / NIGHT CLUB,HOT BODIES CLUB HWY 71 & BURLESON RD,NaN,NaN,NaN,NaN,NaN,NaN
2,FAMILY DISTURBANCE,N,05/02/2008 11:11:00 PM,05/02/2008,2311.0,05/02/2008 11:11:00 PM,05/02/2008,2311.0,RESIDENCE / HOME,9702 CHILSOM TRAIL,NaN,NaN,NaN,NaN,NaN,NaN
3,ASSAULT WITH INJURY,N,03/31/2006 02:50:00 AM,03/31/2006,250.0,03/31/2006 02:50:00 AM,03/31/2006,250.0,RESIDENCE / HOME,721 LAMAR PLACE,NaN,NaN,NaN,NaN,NaN,NaN
4,CRASH/FAIL STOP AND RENDER AID,N,04/09/2008 07:04:00 PM,04/09/2008,1904.0,04/09/2008 09:08:00 PM,04/09/2008,2108.0,STREETS / HWY / ROAD / ALLEY,2500 BLOCK OF GUADALUPE,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2254054,THEFT BY SHOPLIFTING,N,04/05/2020 07:34:00 PM,04/05/2020,1934.0,04/05/2020 07:34:00 PM,04/05/2020,1934.0,DEPARTMENT / DISCOUNT STORE,710 E BEN WHITE SVRD WB,NaN,3111061.0,3111061.0,30.220003,-97.754395,"(30.2200027, -97.75439457)"
2256101,FAMILY DISTURBANCE/PARENTAL,N,04/07/2020 05:02:00 PM,04/07/2020,1702.0,04/07/2020 05:02:00 PM,04/07/2020,1702.0,RESIDENCE / HOME,4600 NUCKOLS CROSS,NaN,NaN,NaN,NaN,NaN,NaN
2257687,THEFT BY SHOPLIFTING,N,04/01/2020 07:12:00 PM,04/01/2020,1912.0,04/01/2020 07:12:00 PM,04/01/2020,1912.0,GAS / SERVICE STATIONS,201 W BEN WHITE BLVD,NaN,3107731.0,3107731.0,30.222510,-97.764874,"(30.22250981, -97.7648738)"
2258266,POSS/PROMO CHILD PORNOGRAPHY,N,05/10/2019 01:00:00 AM,05/10/2019,100.0,05/14/2019 07:45:00 PM,05/14/2019,1945.0,RESIDENCE / HOME,AUSTIN TX 78701,NaN,NaN,NaN,NaN,NaN,NaN


### 1. Date-time

In [9]:
df_date_time = df[['Occurred Date Time', 'Occurred Date', 'Occurred Time', 'Report Date Time', 'Report Date', 'Report Time']]

In [10]:
df[df['Occurred Date Time'].isnull()]

,Highest Offense Description,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,Location Type,Address,Zip Code,X-coordinate,Y-coordinate,Latitude,Longitude,Location
1112,DOC UNREASONABLE NOISE,N,NaN,09/17/2005,NaN,09/17/2005 09:45:00 PM,09/17/2005,2145.0,NaN,5500 TUMBLEWEED DR,NaN,NaN,NaN,NaN,NaN,NaN
2988,COUNTERFEITING,N,NaN,07/09/2003,NaN,07/09/2003 02:09:00 PM,07/09/2003,1409.0,NaN,13376 US HWY 183 N,NaN,NaN,NaN,NaN,NaN,NaN
5412,ENTICING A CHILD,N,NaN,03/24/2006,NaN,03/24/2006 06:13:00 PM,03/24/2006,1813.0,NaN,8800 BLOCK NORTH PLAZA,NaN,0.0,0.0,NaN,NaN,NaN
9556,CRIMINAL TRESPASS,N,NaN,12/01/2004,NaN,12/01/2004 01:24:00 PM,12/01/2004,1324.0,NaN,900 N I 35 SVC RD SB,NaN,NaN,NaN,NaN,NaN,NaN
10739,POSS OF DRUG PARAPHERNALIA,N,NaN,02/13/2004,NaN,02/13/2004 12:54:00 AM,02/13/2004,54.0,NaN,CANTERBURY ST/PLEASANT VALLE,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133193,BURGLARY OF VEHICLE,N,NaN,07/13/2007,NaN,07/13/2007 10:40:00 PM,07/13/2007,2240.0,NaN,4700 E RIVERSIDE DR,78741.0,3122188.0,3122188.0,30.232898,-97.718807,"(30.23289806, -97.71880686)"
2190834,AUTO THEFT,N,NaN,01/16/2003,2400.0,01/17/2003 11:05:00 PM,01/17/2003,2305.0,PARKING LOTS / GARAGE,6855 E US 290 HWY SVRD EB,78723.0,3131259.0,3131259.0,30.320667,-97.687671,"(30.32066659, -97.68767105)"
2216499,DAMAGE CITY PROP,N,NaN,12/13/2006,NaN,12/13/2006 02:19:00 AM,12/13/2006,219.0,NaN,812 SPRINGDALE RD,78702.0,3127908.0,3127908.0,30.258932,-97.699980,"(30.25893229, -97.69997951)"
2220619,BURGLARY OF VEHICLE,N,NaN,05/30/2003,2400.0,05/31/2003 01:58:00 PM,05/31/2003,1358.0,PARKING LOTS / GARAGE,9400 W PARMER LN,78717.0,3105186.0,3105186.0,30.480812,-97.766068,"(30.48081206, -97.76606831)"


In [11]:
df

,Highest Offense Description,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,Location Type,Address,Zip Code,X-coordinate,Y-coordinate,Latitude,Longitude,Location
0,PROTECTIVE ORDER,N,01/23/2012 05:17:00 AM,01/23/2012,517.0,01/23/2012 05:17:00 AM,01/23/2012,517.0,RESIDENCE / HOME,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASSAULT WITH INJURY,N,10/10/2007 09:00:00 PM,10/10/2007,2100.0,10/10/2007 10:00:00 PM,10/10/2007,2200.0,BAR / NIGHT CLUB,HOT BODIES CLUB HWY 71 & BURLESON RD,NaN,NaN,NaN,NaN,NaN,NaN
2,FAMILY DISTURBANCE,N,05/02/2008 11:11:00 PM,05/02/2008,2311.0,05/02/2008 11:11:00 PM,05/02/2008,2311.0,RESIDENCE / HOME,9702 CHILSOM TRAIL,NaN,NaN,NaN,NaN,NaN,NaN
3,ASSAULT WITH INJURY,N,03/31/2006 02:50:00 AM,03/31/2006,250.0,03/31/2006 02:50:00 AM,03/31/2006,250.0,RESIDENCE / HOME,721 LAMAR PLACE,NaN,NaN,NaN,NaN,NaN,NaN
4,CRASH/FAIL STOP AND RENDER AID,N,04/09/2008 07:04:00 PM,04/09/2008,1904.0,04/09/2008 09:08:00 PM,04/09/2008,2108.0,STREETS / HWY / ROAD / ALLEY,2500 BLOCK OF GUADALUPE,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2259643,HARASSMENT,N,01/31/2017 10:31:00 AM,01/31/2017,1031.0,02/23/2017 10:31:00 AM,02/23/2017,1031.0,RESIDENCE / HOME,9802 ROXANNA DR,78748.0,3088221.0,3088221.0,30.176236,-97.827862,"(30.17623555, -97.82786218)"
2259644,CRIMINAL TRESPASS/TRANSIENT,N,03/29/2010 07:17:00 AM,03/29/2010,717.0,03/29/2010 07:17:00 AM,03/29/2010,717.0,CHURCH / SYNAGOGUE / TEMPLE / MOSQUE,203 E 10TH ST,78701.0,3115283.0,3115283.0,30.271040,-97.739653,"(30.27103998, -97.73965326)"
2259645,BURGLARY OF RESIDENCE,N,01/04/2003 06:30:00 PM,01/04/2003,1830.0,01/05/2003 10:31:00 AM,01/05/2003,1031.0,RESIDENCE / HOME,1809 SANCHEZ ST,78702.0,3125587.0,3125587.0,30.281489,-97.706719,"(30.28148881, -97.70671938)"
2259646,BURGLARY NON RESIDENCE,N,03/31/2006 10:46:00 PM,03/31/2006,2246.0,03/31/2006 10:46:00 PM,03/31/2006,2246.0,COMMERCIAL / OFFICE BUILDING,6016 OLD FREDERICKSBURG RD,78749.0,3081299.0,3081299.0,30.234197,-97.848283,"(30.2341966, -97.8482829)"


In [12]:
df['Occurred hour']= pd.to_datetime(df['Occurred Date Time'], format='%m/%d/%Y %I:%M:%S %p').dt.hour
df['Occurred year']= pd.to_datetime(df['Report Date'], format='%m/%d/%Y').dt.year

In [13]:
df['Report hour']= pd.to_datetime(df['Report Date Time'], format='%m/%d/%Y %I:%M:%S %p').dt.hour
df['Report year']= pd.to_datetime(df['Report Date'], format='%m/%d/%Y').dt.year

In [14]:
#df['report_dt'] = pd.to_datetime(df['Report Date Time'], format='%m/%d/%Y %I:%M:%S %p')
df['hour']= pd.to_datetime(df['Report Date Time'], format='%m/%d/%Y %I:%M:%S %p').dt.hour
df['year']= pd.to_datetime(df['Report Date'], format='%m/%d/%Y').dt.year

In [15]:
df[df['Occurred hour'].isnull()].sort_values(by = 'Report year')

,Highest Offense Description,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,Location Type,Address,...,Y-coordinate,Latitude,Longitude,Location,Occurred hour,Occurred year,Report hour,Report year,hour,year
759147,THEFT BY SHOPLIFTING,N,NaN,04/26/2003,2400.0,04/27/2003 12:05:00 AM,04/27/2003,5.0,CONVENIENCE STORE,6105 TECHNI CENTER DR,...,3137612.0,30.275779,-97.668770,"(30.27577941, -97.66876975)",NaN,2003,0.0,2003,0.0,2003
1090693,BURGLARY OF VEHICLE,N,NaN,01/29/2003,2400.0,01/30/2003 06:10:00 AM,01/30/2003,610.0,PARKING LOTS / GARAGE,6501 EAST HILL DR,...,3110506.0,30.345894,-97.752789,"(30.34589433, -97.75278885)",NaN,2003,6.0,2003,6.0,2003
1070548,BURGLARY OF VEHICLE,N,NaN,01/13/2003,2400.0,01/14/2003 11:07:00 AM,01/14/2003,1107.0,PARKING LOTS / GARAGE,1201 BROADMOOR DR,...,3125449.0,30.310440,-97.706369,"(30.31044018, -97.7063688)",NaN,2003,11.0,2003,11.0,2003
1069298,BURGLARY OF VEHICLE,N,NaN,05/07/2003,2400.0,05/08/2003 08:29:00 AM,05/08/2003,829.0,PARKING LOTS / GARAGE,600 BARWOOD PARK,...,3128170.0,30.349500,-97.696676,"(30.34950001, -97.69667616)",NaN,2003,8.0,2003,8.0,2003
1019862,CRIMINAL MISCHIEF,N,NaN,02/14/2003,2400.0,02/15/2003 07:43:00 AM,02/15/2003,743.0,STREETS / HWY / ROAD / ALLEY,9405 MOUNTAIN QUAIL RD,...,3123768.0,30.369685,-97.710087,"(30.36968496, -97.71008695)",NaN,2003,7.0,2003,7.0,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959805,WARRANT ARREST NON TRAFFIC,N,NaN,01/09/2008,NaN,01/09/2008 08:50:00 AM,01/09/2008,850.0,NaN,HOLLY ST / N IH 35 SVRD NB,...,3116389.0,30.257126,-97.736523,"(30.25712587, -97.73652304)",NaN,2008,8.0,2008,8.0,2008
2082409,CRIMINAL MISCHIEF,N,NaN,08/24/2008,NaN,08/24/2008 05:08:00 AM,08/24/2008,508.0,NaN,8801 S IH 35 SVRD NB,...,3102585.0,30.168334,-97.782602,"(30.16833396, -97.78260193)",NaN,2008,5.0,2008,5.0,2008
2101277,DAMAGE CITY PROP,N,NaN,08/02/2008,NaN,08/02/2008 04:17:00 AM,08/02/2008,417.0,NaN,715 E 8TH ST,...,3116884.0,30.267553,-97.734674,"(30.26755343, -97.73467415)",NaN,2008,4.0,2008,4.0,2008
384849,CRIMINAL MISCHIEF,N,NaN,09/13/2010,80.0,09/13/2010 08:26:00 AM,09/13/2010,826.0,PARKING LOTS / GARAGE,2409 GUADALUPE ST,...,3114677.0,30.288215,-97.741112,"(30.28821512, -97.74111227)",NaN,2010,8.0,2010,8.0,2010


In [16]:
del df['Occurred Date Time']
del df['Occurred Time']
del df['Report Date Time']
del df['Report Time']


In [17]:
#df['Report Time'] = df['Report Time'].apply(lambda x: 0 if pd.isnull(x) else x)


### 2. Location Type

In [18]:
df['Location Type'] = df['Location Type'].apply(lambda x: 'OTHER / UNKNOWN' if pd.isnull(x) else x)

### 3. Address

In [19]:
df[df['Address'].isnull()]

,Highest Offense Description,Family Violence,Occurred Date,Report Date,Location Type,Address,Zip Code,X-coordinate,Y-coordinate,Latitude,Longitude,Location,Occurred hour,Occurred year,Report hour,Report year,hour,year
2574,FRAUD - OTHER,N,11/20/2009,12/13/2009,RESIDENCE / HOME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,2009,18.0,2009,18.0,2009
3788,CUSTODY ARREST TRAFFIC WARR,N,01/16/2006,01/16/2006,STREETS / HWY / ROAD / ALLEY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,2006,20.0,2006,20.0,2006
3908,WARRANT ARREST NON TRAFFIC,N,12/07/2010,12/07/2010,STREETS / HWY / ROAD / ALLEY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,2010,18.0,2010,18.0,2010
6743,COUNTERFEITING,N,05/12/2005,05/30/2005,RESIDENCE / HOME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,2005,13.0,2005,13.0,2005
7804,CRIMINAL TRESPASS,N,01/20/2007,01/20/2007,TRANSPORTATION (AIR / BUS / TRAIN - TERMINALS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,2007,19.0,2007,19.0,2007
9384,CRIMINAL MISCHIEF,N,09/22/2005,09/23/2005,RESIDENCE / HOME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,2005,21.0,2005,21.0,2005
9843,WARRANT ARREST NON TRAFFIC,N,07/21/2006,07/21/2006,STREETS / HWY / ROAD / ALLEY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2006,9.0,2006,9.0,2006
10786,HARASSMENT,N,05/18/2008,05/18/2008,RESIDENCE / HOME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2008,1.0,2008,1.0,2008
13068,BURGLARY NON RESIDENCE,N,04/03/2009,04/09/2009,COMMERCIAL / OFFICE BUILDING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,2009,13.0,2009,13.0,2009
1057997,POSS CONTROLLED SUB/NARCOTIC,N,04/18/2005,04/18/2005,OTHER / UNKNOWN,NaN,NaN,3121801.0,3121801.0,30.269700,-97.719036,"(30.26969963, -97.71903643)",14.0,2005,14.0,2005,14.0,2005


### 4. Zip Code

In [20]:
df['Zip Code'] = df['Zip Code'].apply(lambda x: str(x)[:5])

In [21]:
df[df['Zip Code'] == '0.0'].sort_values(by = 'Occurred year')

,Highest Offense Description,Family Violence,Occurred Date,Report Date,Location Type,Address,Zip Code,X-coordinate,Y-coordinate,Latitude,Longitude,Location,Occurred hour,Occurred year,Report hour,Report year,hour,year
1601758,CUSTODY ARREST TRAFFIC WARR,N,10/30/2003,10/30/2003,OTHER / UNKNOWN,414 KRAUSE SPRINGS RD,0.0,2984859.0,2984859.0,30.478606,-98.148203,"(30.47860645, -98.14820274)",14.0,2003,14.0,2003,14.0,2003
1283349,CUSTODY ARREST TRAFFIC WARR,N,12/26/2003,12/26/2003,STREETS / HWY / ROAD / ALLEY,11600 BLOCK N FM 620 RD,0.0,3086996.0,3086996.0,30.456737,-97.824457,"(30.45673748, -97.82445738)",23.0,2003,23.0,2003,23.0,2003
1673524,FAILURE TO IDENTIFY,N,06/05/2003,06/05/2003,STREETS / HWY / ROAD / ALLEY,800 INDIAN MOUND RD,0.0,2998522.0,2998522.0,30.473163,-98.104946,"(30.47316308, -98.10494576)",17.0,2003,17.0,2003,17.0,2003
1724760,BURGLARY OF VEHICLE,N,02/01/2004,02/01/2004,COMMERCIAL / OFFICE BUILDING,101 ELIZABETH DR,0.0,2985563.0,2985563.0,30.517440,-98.145085,"(30.5174404, -98.14508531)",19.0,2004,19.0,2004,19.0,2004
1620170,VIOL OF PARK CURFEW,N,08/14/2004,08/14/2004,GOVERNMENT / PUBLIC BUILDING,400 RUSTIC LN,0.0,2995332.0,2995332.0,30.474933,-98.115033,"(30.47493284, -98.11503339)",22.0,2004,22.0,2004,22.0,2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265698,CRIMINAL MISCHIEF,N,11/05/2008,11/05/2008,RESIDENCE / HOME,3601 MCNEIL RD,0.0,3122367.0,3122367.0,30.467283,-97.711878,"(30.46728279, -97.71187785)",1.0,2008,1.0,2008,1.0,2008
5578,INDECENCY WITH A CHILD/CONTACT,Y,04/21/2008,04/21/2008,RESIDENCE / HOME,11000 BLOCK MARBLE RD,0.0,0.0,0.0,NaN,NaN,NaN,14.0,2008,14.0,2008,14.0,2008
1730951,BURGLARY OF RESIDENCE,N,01/19/2008,01/23/2009,RESIDENCE / HOME,12223 OLD STAGE TRL,0.0,3094952.0,3094952.0,30.454548,-97.799257,"(30.45454835, -97.79925719)",17.0,2009,3.0,2009,3.0,2009
1216963,HARASSMENT,N,12/30/2008,02/17/2009,RESIDENCE / HOME,10507 MELLOW MEADOWS,0.0,3094838.0,3094838.0,30.460815,-97.799454,"(30.46081473, -97.79945431)",20.0,2009,12.0,2009,12.0,2009


In [22]:
df[df['Zip Code'] == '0.0'].sort_values(by = 'Occurred year').count()

Highest Offense Description    62
Family Violence                62
Occurred Date                  62
Report Date                    62
Location Type                  62
Address                        62
Zip Code                       62
X-coordinate                   62
Y-coordinate                   62
Latitude                       61
Longitude                      61
Location                       61
Occurred hour                  62
Occurred year                  62
Report hour                    62
Report year                    62
hour                           62
year                           62
dtype: int64

Just 62 rows and all before 2009 year. Since they aren't recent data we can get rid of them.

In [23]:
df = df[df['Zip Code'] != '0.0']

In [24]:
df = df[df['Zip Code'] != 'nan']

In [25]:
df[df['Zip Code'] == 'nan'].sort_values(by = 'Occurred year')

,Highest Offense Description,Family Violence,Occurred Date,Report Date,Location Type,Address,Zip Code,X-coordinate,Y-coordinate,Latitude,Longitude,Location,Occurred hour,Occurred year,Report hour,Report year,hour,year


In [26]:
del df['X-coordinate']           
del df['Y-coordinate']             
del df['Latitude']                   
del df['Longitude']                      
del df['Location']                    

In [27]:
df.isnull().sum()*100/df.isnull().count()

Highest Offense Description    0.000000
Family Violence                0.000000
Occurred Date                  0.000000
Report Date                    0.000000
Location Type                  0.000000
Address                        0.000000
Zip Code                       0.000000
Occurred hour                  0.006768
Occurred year                  0.000000
Report hour                    0.000045
Report year                    0.000000
hour                           0.000045
year                           0.000000
dtype: float64

In [28]:
df.dropna()

,Highest Offense Description,Family Violence,Occurred Date,Report Date,Location Type,Address,Zip Code,Occurred hour,Occurred year,Report hour,Report year,hour,year
9,RAPE,N,09/11/2018,09/11/2018,RESIDENCE / HOME,14000 BLOCK HYMILL DR,78660,19.0,2018,23.0,2018,23.0,2018
15,INDECENCY WITH A CHILD/CONTACT,N,10/29/2007,10/29/2007,RESIDENCE / HOME,200 BLOCK TIFFANY TRL,78719,16.0,2007,16.0,2007,16.0,2007
22,ASSAULT CONTACT-SEXUAL NATURE,Y,02/24/2019,02/26/2019,RESIDENCE / HOME,8600 BLOCK AXIS DR,78749,21.0,2019,16.0,2019,16.0,2019
45,CAMPING IN PARK,N,02/18/2008,02/18/2008,STREETS / HWY / ROAD / ALLEY,700 E 4TH ST,78701,9.0,2008,9.0,2008,9.0,2008
64,CRIMINAL TRESPASS,N,07/18/2003,07/18/2003,FIELD / WOODS,3427 PARKER LN,78741,5.0,2003,5.0,2003,5.0,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2259643,HARASSMENT,N,01/31/2017,02/23/2017,RESIDENCE / HOME,9802 ROXANNA DR,78748,10.0,2017,10.0,2017,10.0,2017
2259644,CRIMINAL TRESPASS/TRANSIENT,N,03/29/2010,03/29/2010,CHURCH / SYNAGOGUE / TEMPLE / MOSQUE,203 E 10TH ST,78701,7.0,2010,7.0,2010,7.0,2010
2259645,BURGLARY OF RESIDENCE,N,01/04/2003,01/05/2003,RESIDENCE / HOME,1809 SANCHEZ ST,78702,18.0,2003,10.0,2003,10.0,2003
2259646,BURGLARY NON RESIDENCE,N,03/31/2006,03/31/2006,COMMERCIAL / OFFICE BUILDING,6016 OLD FREDERICKSBURG RD,78749,22.0,2006,22.0,2006,22.0,2006


In [29]:
#pip install missingno

In [ ]:
import missingno as msno
#missing values
df.isnull().sum()*100/df.isnull().count()

# Create nullity matrix for missing values 

msno.matrix(df)

In [ ]:
df.replace([np.inf, -np.inf], np.nan)
df.dropna(axis=0,  inplace=True)

# EDA

### Crime per year

In [ ]:
crimes_per_year = df['year'].value_counts().sort_index()

g = sns.barplot(x=crimes_per_year.index, y=crimes_per_year.values, color='b')
g.set_xticklabels(g.get_xticklabels(),rotation=90)
g.set(xlabel='Year', ylabel='Crimes reported')
plt.title('Number of reported crimes per year')
plt.show()

In [ ]:
for i in np.sort(df.year.unique()):
    print(i)
    print(df[df.year==i].groupby('Highest Offense Description').year.count().sort_values(ascending=False).head(5))



In [ ]:
df[df['Highest Offense Description'].\
   isin(['BURGLARY OF VEHICLE', 'THEFT', 'CRIMINAL MISCHIEF','FAMILY DISTURBANCE', 'THEFT BY SHOPLIFTING', 'BURGLARY OF RESIDENCE'])].groupby(['year', 'Highest Offense Description'])['Zip Code'].count().sort_values(ascending=False).unstack().plot(kind='bar', legend='best', stacked=True)
plt.legend(loc='best', prop={'size': 8}, bbox_to_anchor=(1.1, 1.05))


### The most dangerous Zip Codes

In [ ]:
#top 5 zip codes 
df.fillna(0, inplace=True)
df['Zip Code'] = df['Zip Code'].apply(lambda x: str(x)[:5])
Top5 = df[df['Zip Code']!='0.0']['Zip Code'].value_counts().sort_values(ascending=False).head(5)
Top5.plot(kind='bar')
plt.xlabel("Zip Codes")
plt.ylabel("Amount of crimes")

plt.title('Zip Codes with the highest amount of crime')

# Highest Offense Description

In [ ]:
len(df['Highest Offense Description'].unique())

In [ ]:
children_safety = ['RUNAWAY CHILD', 'CHILD ENDANGERMENT- ABANDONMEN', 'CHILD CUSTODY INTERFERE', 'POSS OF ALCOHOL - AGE 17 TO 20','POSS/PROMO CHILD PORNOGRAPHY', 'INJ/CHILD FV (NO CARE/CUSTODY)','SALE OR PURCHASE OF CHILD','DISRUPTIVE ACTS AT SCHOOLS','FIREARMS ON SCHOOL PROP','FAILURE TO REPORT CHILD ABUSE', 'EXPIRED-SOLICITATION OF CHILD', 'LOITERING ON SCHOOL PROP', 'ASSAULT - SCHOOL PERSONNEL',]
murder = ['HARASSMENT',  'ASSAULT WITH INJURY', 'ASSAULT ON PEACE OFFICER', 'MURDER','MANSLAUGHTER','CRASH/NEGLIGENT HOMICIDE', 'ARSON WITH BODILY INJURY', 'CRASH/MANSLAUGHTER','EXPIRED - DEADLY ASSAULT', 'EXPIRED-ATT CAPITAL MURDER', 'CRASH/MURDER', 'EXPIRED-ATT CAPITAL MURDER' ]
burglary_robbery = [ 'THEFT OF BICYCLE', 'BURGLARY OF VEHICLE', 'THEFT FROM BUILDING', 'THEFT FROM PERSON', 'IDENTITY THEFT','THEFT OF AUTO PARTS', 'ROBBERY BY THREAT', 'THEFT CATALYTIC CONVERTER',
       'IDENTITY THEFT-TAX RETURNS', 'THEFT OF HEAVY EQUIPMENT','EXPIRED-ATT ROBBERY BY ASSAULT','THEFT BY EMBEZZLEMENT','POCKET PICKING','THEFT/TILL TAPPING']
safety = ['THEFT','SUSPICIOUS PERSON',  'AUTO THEFT', 'BURGLARY OF RESIDENCE','FAMILY DISTURBANCE',   'FAMILY DISTURBANCE/PARENTAL','ROBBERY BY ASSAULT', 'TERRORISTIC THREAT',
       'AGG ROBBERY/DEADLY WEAPON',  'THEFT BY SHOPLIFTING',  'BURGLARY OF COIN-OP MACHINE', 'FORGERY AND PASSING',
     
      'UNLAWFUL CARRYING WEAPON','VOCO - ALCOHOL  CONSUMPTION', 'DOC FIGHTING','AGG ASSAULT','ASSAULT BY THREAT', 'THEFT OF LICENSE PLATE','GRAFFITI','PROSTITUTION', 'POSS OF PROHIBITED WEAPON','MAIL THEFT',  'STALKING', 'VIOL OF PARK CURFEW','URINATING IN PUBLIC PLACE', 'THEFT- APPROPRIATE STOLEN PROP', 'DOC WINDOW PEEPING-RESIDENCE','KIDNAPPING', 'WEAPON VIOL - OTHER','DOC DISCHARGE GUN - PUB PLACE', 'AMPLIFIED MUSIC / VEHICLE','ATT THEFT', 'PIGEON DROP', 'PROMOTION OF PROSTITUTION','TAKE WEAPON FRM POLICE OFFICER','DEADLY CONDUCT FAM/DATE VIOL','ATT AUTO THEFT','SEXTING/TRANSMIT SEXUAL PHOTOS','KIDNAPPING FAM VIO','AGG KIDNAPPING FAM VIO','RIOT','DOMESTIC VIOLENCE/ALARM', 'PURCHASING PROSTITUTION','DOC WINDOW PEEPING - PUB AREA','ASSAULT OF A PREGNANT WOMAN','SEXUAL COERCION', 'POSS CRIMINAL INSTRUMENT', 'FAIL DISPLAY HANDGUN LICENSE', 'EXPIRED-ATT MURDER', 'INJURY TO ELDERLY PERSON - SBI']
drugs = [ 'POSSESSION OF MARIJUANA','POSS OF DRUG PARAPHERNALIA']
rape = ['RAPE', 'INDECENCY WITH A CHILD/CONTACT',
       'ASSAULT  CONTACT-SEXUAL NATURE','STATUTORY RAPE OF CHILD',
       'SEXUAL ASSAULT OF CHILD/OBJECT','AGG SEXUAL ASSAULT CHILD/OBJEC', 'AGG RAPE',]

In [ ]:
top5_zip = df[(df['Highest Offense Description'].isin(['CAPITAL MURDER', 'MURDER', 'RAPE', 'AGG RAPE OF A CHILD', 'STATUTORY RAPE OF CHILD', \
                                                                                          'AGG RAPE', 'RAPE OF A CHILD']) ) ]['Zip Code'].value_counts().sort_values(ascending=False).head(10).index

In [ ]:
top5_zip

In [ ]:
df[['Location Type', 'Highest Offense Description']][df['Highest Offense Description'] ==  'AGG RAPE OF A CHILD'].groupby('Location Type').count()

In [ ]:
df['Highest Offense Description'].value_counts().head()

In [ ]:
df['Highest Offense Description'].value_counts().head(20).plot(kind='bar')

plt.xlabel("Highest Offense Description")
plt.ylabel("Amount of crimes")
plt.title('20 the most committed crimes')

# Population

In [ ]:
temp = df.groupby(['Zip Code', 'year'])['Location Type'].count().sort_values(ascending=False).reset_index()
df2 = temp.pivot("Zip Code", "year", "Location Type")
df2 = df2.fillna(0)

In [ ]:
df2.loc['78613'].plot()

In [ ]:
df2.loc['78701'].plot()

In [ ]:
df2.loc['78754'].plot()

In [ ]:
df_mean = df2[[2015, 2016, 2017, 2018, 2019]]

In [ ]:
df_mean['mean_crime'] = df_mean.mean(axis=1)

In [ ]:
df_mean

In [ ]:
#df2['zipcode'] = df2.index
df_mean = df_mean.reset_index()

In [ ]:
df_mean.rename(columns = {'Zip Code':'zipcode'}, inplace = True)

In [ ]:
del df_mean[2015]
del df_mean[2016]
del df_mean[2017]
del df_mean[2018]
del df_mean[2019]

In [ ]:
df_mean['zipcode']= df_mean['zipcode'].astype(int) 

In [ ]:
df_population = pd.read_csv('Desktop/Project Austin/Texas Population.csv')

In [ ]:
df_population = df_population.dropna()

In [ ]:
df_population.info()

In [ ]:
df_population['Population']= df_population['Population'].astype(str) 

In [ ]:
df_population['Population'] = df_population['Population'].apply(lambda x: x.replace('.0', ''))

In [ ]:
df_population['Population']= df_population['Population'].astype(int) 
df_population['Zip Code']= df_population['Zip Code'].astype(int) 

In [ ]:
df_population.info()

In [ ]:
df_population.rename(columns = {'Zip Code':'zipcode'}, inplace = True)

In [ ]:
df_ratio = pd.merge(df_mean, df_population, on='zipcode')


In [ ]:
df_ratio['rate'] = df_ratio['mean_crime'] / df_ratio['Population'] * 10000

In [ ]:
df_ratio.sort_values(by = 'rate', ascending = False).head(15)

In [ ]:
df_ratio.to_csv('Desktop/Project Austin/Austin_Crime_Rate.csv')